## Task 3: Pizza Sales Analysis

### Loading the csv files.

In [ ]:
import pandas as pd # importing pandas

import matplotlib.pyplot as plt # importing matplotlib

orders = pd.read_csv('../Downloads/Pizza+Place+Sales/pizza_sales/orders.csv')
order_details = pd.read_csv('../Downloads/Pizza+Place+Sales/pizza_sales/order_details.csv')
pizzas = pd.read_csv('../Downloads/Pizza+Place+Sales/pizza_sales/pizzas.csv')
pizza_types = pd.read_csv('../Downloads/Pizza+Place+Sales/pizza_sales/pizza_types.csv', encoding="latin-1") 


In [ ]:
orders.head() # To view orders

,order_id,date,time
0,1,2015-01-01,11:38:36
1,2,2015-01-01,11:57:40
2,3,2015-01-01,12:12:28
3,4,2015-01-01,12:16:31
4,5,2015-01-01,12:21:30


In [9]:
order_details.head() # To view order_details

,order_details_id,order_id,pizza_id,quantity
0,1,1,hawaiian_m,1
1,2,2,classic_dlx_m,1
2,3,2,five_cheese_l,1
3,4,2,ital_supr_l,1
4,5,2,mexicana_m,1


In [ ]:
pizzas.head()

In [ ]:
pizza_types.head()

## Merging of the datasets

In [33]:
# Merging order_details to pizzas to get prices
details_prices = pd.merge(order_details,pizzas, on="pizza_id")

# Merging with orders to get date/time
full_data = pd.merge(details_prices, orders, on= "order_id")

# Merging with pizza_types to get pizza category and ingredients
full_data = pd.merge(full_data, pizza_types, on= "pizza_type_id")

# Converting data/time columns
full_data["date"] = pd.to_datetime(full_data["date"])
full_data["time"] = pd.to_datetime(full_data["time"]).dt.time


C:\Users\Hp\AppData\Local\Temp\ipykernel_6700\1322273196.py:12: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  full_data["time"] = pd.to_datetime(full_data["time"]).dt.time


In [35]:
full_data.head()

,order_details_id,order_id,pizza_id,quantity,pizza_type_id,size,price,date,time,name,category,ingredients
0,1,1,hawaiian_m,1,hawaiian,M,13.25,2015-01-01,11:38:36,The Hawaiian Pizza,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese"
1,2,2,classic_dlx_m,1,classic_dlx,M,16.00,2015-01-01,11:57:40,The Classic Deluxe Pizza,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,..."
2,3,2,five_cheese_l,1,five_cheese,L,18.50,2015-01-01,11:57:40,The Five Cheese Pizza,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go..."
3,4,2,ital_supr_l,1,ital_supr,L,20.75,2015-01-01,11:57:40,The Italian Supreme Pizza,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni..."
4,5,2,mexicana_m,1,mexicana,M,16.00,2015-01-01,11:57:40,The Mexicana Pizza,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O..."


In [20]:
full_data.duplicated().sum()

np.int64(0)

In [36]:
full_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48620 entries, 0 to 48619
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   order_details_id  48620 non-null  int64         
 1   order_id          48620 non-null  int64         
 2   pizza_id          48620 non-null  object        
 3   quantity          48620 non-null  int64         
 4   pizza_type_id     48620 non-null  object        
 5   size              48620 non-null  object        
 6   price             48620 non-null  float64       
 7   date              48620 non-null  datetime64[ns]
 8   time              48620 non-null  object        
 9   name              48620 non-null  object        
 10  category          48620 non-null  object        
 11  ingredients       48620 non-null  object        
dtypes: datetime64[ns](1), float64(1), int64(3), object(7)
memory usage: 4.5+ MB


### 1. What is the total revenue/sales ?

In [37]:
# Calculating the revenue for each other item
full_data['item_revenue'] = full_data['price'] * full_data['quantity']
# Calculating the total revenue
total_revenue = full_data['item_revenue'].sum()

print(f"The total revenue is: {total_revenue:.2f}")

The total revenue is: 817860.05


### 2.Find the total quantity sold

In [38]:
# To calculate the total quantity
total_quantity_sold = full_data['quantity'].sum()

# Print the total quantity sold
print(f"The total quantity of pizzas sold is: {total_quantity_sold}")

The total quantity of pizzas sold is: 49574


### 3. Find the total orders

In [39]:
# We use the unique order IDs to find and sum the total orders
total_orders = full_data['order_id'].nunique()

# Print the total number of orders
print(f"The total number of orders is: {total_orders}")

The total number of orders is: 21350


### 4. How many pizza types do they sell?

In [40]:
# To  calculate the sypes of pizza, we find the number of unique pizza type IDs
number_of_pizza_types = full_data['pizza_type_id'].nunique()

# Print the total number of pizza types
print(f"The total number of pizza types they sell is: {number_of_pizza_types}")

The total number of pizza types they sell is: 32


### 5.	Find the average price of the pizzas.

In [31]:
# To calculate the average price of the pizzas
average_price = full_data['price'].mean()

# Print the average price
print(f"The average price of the pizzas is: ${average_price:.2f}")

The average price of the pizzas is: $16.49


### 6. What are the peak hours of sale?

In [41]:
# To extract the hour from the time column
full_data['hour'] = full_data['time'].apply(lambda x: x.hour)

# To count and group the number of orders by per hour
sales_by_hour = full_data.groupby('hour')['order_id'].count()

# To identify the peak hours
peak_sales_count = sales_by_hour.max()
peak_hours = sales_by_hour[sales_by_hour == peak_sales_count].index.tolist()

# Format the peak hours for better readability
peak_hours_formatted = [f"{hour:02d}:00 - {(hour + 1):02d}:00" for hour in peak_hours]

print(f"The peak hour(s) of sales are: {', '.join(peak_hours_formatted)}")
print(f"Number of orders during peak hour(s): {peak_sales_count}")

The peak hour(s) of sales are: 12:00 - 13:00
Number of orders during peak hour(s): 6543


### 7. Find the total sales made on each day of the week. Which day of the week is when sales are made the most?

In [42]:
# To get the day of the week
full_data['day_of_week'] = full_data['date'].dt.day_name()

# To get the revenue(although i already did earlier)
full_data['item_revenue'] = full_data['price'] * full_data['quantity']

# To group by day of the week and sum the revenue
sales_by_day = full_data.groupby('day_of_week')['item_revenue'].sum()

# To order the days of the week
days_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
sales_by_day = sales_by_day.reindex(days_order)

# To find the day with the most sales
peak_sales_day = sales_by_day.idxmax()
peak_sales_amount = sales_by_day.max()

print("Total sales for each day of the week:")
print(sales_by_day)
print(f"\nThe day with the most sales is: {peak_sales_day} with a total of ${peak_sales_amount:.2f}")


Total sales for each day of the week:
day_of_week
Monday       107329.55
Tuesday      114133.80
Wednesday    114408.40
Thursday     123528.50
Friday       136073.90
Saturday     123182.40
Sunday        99203.50
Name: item_revenue, dtype: float64

The day with the most sales is: Friday with a total of $136073.90


### 8.	Find the top 5 bestselling pizzas.

In [49]:
print(pizzas.columns)

# Convert the Series to a DataFrame
top_5_pizza_counts_df = top_5_pizza_counts.reset_index()
top_5_pizza_counts_df.columns = ['pizza_id', 'order_count']

# Merge with the 'pizzas' DataFrame to get 'pizza_type_id'
top_5_pizzas_with_type = pd.merge(top_5_pizza_counts_df, pizzas, on='pizza_id')

# Merge with the 'pizza_types' DataFrame to get the pizza names
top_5_bestselling_pizzas = pd.merge(top_5_pizzas_with_type, pizza_types, on='pizza_type_id')

# Select and order the desired columns
top_5_bestselling_pizzas = top_5_bestselling_pizzas[['pizza_id', 'name', 'order_count']]
top_5_bestselling_pizzas.rename(columns={'name': 'pizza_name'}, inplace=True)

# Print the top 5 bestselling pizzas
print("Top 5 Bestselling Pizzas:")
print(top_5_bestselling_pizzas)

Index(['pizza_id', 'pizza_type_id', 'size', 'price'], dtype='object')
Top 5 Bestselling Pizzas:
        pizza_id                pizza_name  order_count
0     big_meat_s        The Big Meat Pizza         1811
1     thai_ckn_l    The Thai Chicken Pizza         1365
2  five_cheese_l     The Five Cheese Pizza         1359
3  four_cheese_l     The Four Cheese Pizza         1273
4  classic_dlx_m  The Classic Deluxe Pizza         1159


### 9.	Find the sales made in each month. Any trend noticeable?

In [50]:
# 1. Extract the month from the 'date' column
full_data['month'] = full_data['date'].dt.month

# 2. Calculate the total revenue for each month
sales_by_month = full_data.groupby('month')['item_revenue'].sum()

# 3. Present the sales by month with month names for better readability
month_names = pd.to_datetime(sales_by_month.index, format='%m').strftime('%B')
sales_by_month_named = pd.Series(sales_by_month.values, index=month_names)

print("Total Sales by Month:")
print(sales_by_month_named)

# 4. Analyze for trends (this will be based on the output)
print("\nTrend Analysis:")
if len(sales_by_month) > 2:
    # You can implement more sophisticated trend analysis here,
    # but for a basic check, we can look for increasing or decreasing patterns.
    sales_diff = sales_by_month.diff().dropna()
    if all(sales_diff > 0):
        print("There is a general increasing trend in sales over the months.")
    elif all(sales_diff < 0):
        print("There is a general decreasing trend in sales over the months.")
    else:
        print("The sales trend appears to be fluctuating across the months.")
else:
    print("Insufficient data to determine a clear trend.")

    # To find the month with the highest sales
if not sales_by_month.empty:
    peak_month = sales_by_month.idxmax()
    peak_month_name = pd.to_datetime([peak_month], format='%m').strftime('%B')[0]
    peak_sales = sales_by_month.max()
    print(f"\nThe month with the highest sales is {peak_month_name} with a total of ${peak_sales:.2f}.")
else:
    print("\nNo sales data available to determine the peak month.")


Total Sales by Month:
month
January      69793.30
February     65159.60
March        70397.10
April        68736.80
May          71402.75
June         68230.20
July         72557.90
August       68278.25
September    64180.05
October      64027.60
November     70395.35
December     64701.15
dtype: float64

Trend Analysis:
The sales trend appears to be fluctuating across the months.

The month with the highest sales is July with a total of $72557.90.


### 10.	Are there pizza types that are not doing well on the menu?

In [54]:
# To calculate the total revenue for each pizza type ID
pizza_type_revenue = full_data.groupby('pizza_type_id')['item_revenue'].sum().sort_values(ascending=True)

# To identify the pizza types with the lowest revenue (e.g., bottom 5)
lowest_revenue_pizza_types_ids = pizza_type_revenue.head()

# To merge with 'pizza_types' to get the names
lowest_revenue_pizza_types = pd.merge(lowest_revenue_pizza_types_ids.reset_index(), pizza_types, left_on='pizza_type_id', right_on='pizza_type_id')
lowest_revenue_pizza_types.rename(columns={'pizza_type_id': 'pizza_type_id', 'item_revenue': 'total_revenue', 'name': 'pizza_name'}, inplace=True)

print("\nPizza Types with the Lowest Total Revenue:")
print(lowest_revenue_pizza_types[['pizza_type_id', 'pizza_name', 'total_revenue']])


Pizza Types with the Lowest Total Revenue:
  pizza_type_id                 pizza_name  total_revenue
0    brie_carre       The Brie Carre Pizza       11588.50
1  green_garden     The Green Garden Pizza       13955.75
2  spinach_supr  The Spinach Supreme Pizza       15277.75
3  mediterraneo    The Mediterranean Pizza       15360.50
4    spin_pesto    The Spinach Pesto Pizza       15596.00


### Exploring the data
### Are there any pizza sizes that are consistently underperforming for specific pizza types?

In [55]:
# To group by pizza_type_id and size and count orders
size_performance = full_data.groupby(['pizza_type_id', 'size'])['order_id'].count().reset_index()
size_performance.rename(columns={'order_id': 'order_count'}, inplace=True)

# To merge with 'pizza_types' to get pizza names
size_performance_with_names = pd.merge(size_performance, pizza_types, on='pizza_type_id')
size_performance_with_names.rename(columns={'name': 'pizza_name'}, inplace=True)

# To analyze for underperforming sizes within each pizza type
def identify_underperforming_sizes(df):
    total_orders = df['order_count'].sum()
    # To define a threshold for underperformance (e.g., less than 10% of the total orders for this pizza type)
    threshold = total_orders * 0.10
    underperforming = df[df['order_count'] < threshold]
    return underperforming[['size', 'order_count']]

underperforming_sizes = size_performance_with_names.groupby('pizza_name').apply(identify_underperforming_sizes).reset_index(level=0)
underperforming_sizes.rename(columns={'pizza_name': 'pizza_type'}, inplace=True)

print("Underperforming Pizza Sizes for Specific Pizza Types (based on order count):")
if not underperforming_sizes.empty:
    print(underperforming_sizes)
else:
    print("No consistently underperforming pizza sizes found based on the current threshold.")

Underperforming Pizza Sizes for Specific Pizza Types (based on order count):
                   pizza_type size  order_count
13  The Chicken Alfredo Pizza    S           96
87            The Greek Pizza  XXL           28
23     The Green Garden Pizza    L           94


C:\Users\Hp\AppData\Local\Temp\ipykernel_6700\2832365760.py:17: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  underperforming_sizes = size_performance_with_names.groupby('pizza_name').apply(identify_underperforming_sizes).reset_index(level=0)


### What are the most popular pizza combinations (size + type)?

In [56]:
# To group by pizza_type_id and size and count the number of orders
popular_combinations = full_data.groupby(['pizza_type_id', 'size'])['order_id'].count().sort_values(ascending=False).reset_index()
popular_combinations.rename(columns={'order_id': 'order_count'}, inplace=True)

# To merge with 'pizza_types' to get pizza names
popular_combinations_with_names = pd.merge(popular_combinations, pizza_types, on='pizza_type_id')
popular_combinations_with_names.rename(columns={'name': 'pizza_name'}, inplace=True)

# To display the most popular combinations (e.g., top 5 or top 10)
top_popular_combinations = popular_combinations_with_names.head(10) # You can adjust the number here

print("Most Popular Pizza Combinations (Size + Type):")
print(top_popular_combinations[['pizza_name', 'size', 'order_count']])

Most Popular Pizza Combinations (Size + Type):
                    pizza_name size  order_count
0           The Big Meat Pizza    S         1811
1       The Thai Chicken Pizza    L         1365
2        The Five Cheese Pizza    L         1359
3        The Four Cheese Pizza    L         1273
4     The Classic Deluxe Pizza    M         1159
5      The Spicy Italian Pizza    L         1088
6           The Hawaiian Pizza    S         1001
7  The Southwest Chicken Pizza    L          993
8   The Barbecue Chicken Pizza    L          967
9   The Barbecue Chicken Pizza    M          926
